In [ ]:
import json
import shutil

import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import pandas as pd
import pdal
import pybabylonjs
from pybabylonjs import Show as show
import tiledb

In [ ]:
#!wget -nc "https://github.com/PDAL/data/blob/master/workshop/autzen.laz?raw=true" -O "autzen.laz"

In [ ]:
#try:
#    shutil.rmtree("autzen")
#    pass
#except:
#    pass

In [ ]:
pipeline1 = (
  pdal.Reader("autzen.laz") |
  pdal.Filter.stats() |
  pdal.Writer.tiledb(array_name="autzen1",chunk_size=10000000)
)

# count = pipeline1.execute()  

In [ ]:
pipeline2 = (
  pdal.Reader("autzen.laz") | 
  pdal.Writer.tiledb(array_name="autzen2",x_tile_size=10, y_tile_size=10, z_tile_size=1,chunk_size=10000000)
)

# count = pipeline2.execute() 

In [ ]:
pipeline3 = (
  pdal.Reader("autzen.laz") |
  pdal.Filter.stats() |
  pdal.Writer.tiledb(array_name="autzen3",chunk_size=10000000,data_tile_capacity=100000)
)

# count = pipeline3.execute()  

In [ ]:
pipeline4 = (
  pdal.Reader("autzen.laz") |
  pdal.Filter.stats() |
  pdal.Writer.tiledb(array_name="autzen4",chunk_size=100000,data_tile_capacity=100000)
)

# count = pipeline4.execute()  

In [ ]:
pipeline5 = (
  pdal.Reader("autzen.laz") |
  pdal.Filter.stats() |
  pdal.Writer.tiledb(array_name="autzen5",chunk_size=100000)
)

# count = pipeline5.execute()  

## Point cloud

In [ ]:
%%time
with tiledb.open("autzen1") as arr:
    df = arr.df[636800:637800, 851000:853000, 406.14:615.26]

df.head()

In [ ]:
data = {
    'X': df['X'],
    'Y': df['Y'],
    'Z': df['Z'],
    'Red': df['Red'] / 255.0,
    'Green': df['Green'] / 255.0,
    'Blue': df['Blue'] / 255.0
}

In [ ]:
show.from_dict(data=data,
                style = 'pointcloud',
                width = 800,
                height = 600,
                inspector = True)

In [ ]:
babylon = pybabylonjs.BabylonJS()
babylon.value = data
babylon.z_scale = .2
babylon

## MBRS

### Default sorting

In [ ]:
show.from_array(array_uri='autzen1',
                style='mbrs',
                width=800,
                height=600,
                z_scale = 0.5,
                wheel_precision = 50)

### Optimized sorting - visual tuning - 2.5 x faster loading of data from array

In [ ]:
show.from_array(array_uri='autzen5',
                style='mbrs',
                width=800,
                height=600,
                z_scale = 0.2,
                wheel_precision = 50)

In [ ]:
%%time
with tiledb.open("autzen1") as arr:
    df = arr.df[636800:637800, 851000:853000, 406.14:615.26]

In [ ]:
%%time
with tiledb.open("autzen5") as arr:
    df = arr.df[636800:637800, 851000:853000, 406.14:615.26]

## 4D 

In [ ]:
%%time
with tiledb.open("autzen5") as arr:
    df = pd.DataFrame(arr[636800:637800, 851000:853000, 406.14:615.26])
    
df.sort_values(by=["GpsTime"], inplace=True)

data = {
    'X': df['X'],
    'Y': df['Y'],
    'Z': df['Z'],
    'Red': df['Red'] / 255.0,
    'Green': df['Green'] / 255.0,
    'Blue': df['Blue'] / 255.0,
    'GpsTime': df['GpsTime']}

In [ ]:
show.from_dict(data=data, style="pointcloud", time=True)